In [1]:
%load_ext autoreload
%autoreload 2

In [164]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [165]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.group_based_policies import *
from patient.ordering_policies import *
from patient.provider_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [306]:
if is_jupyter: 
    seed        = 43
    num_patients = 5
    num_providers = 5
    provider_capacity = 1
    top_choice_prob = 0.75
    true_top_choice_prob = 0.75
    choice_model = "uniform_choice"
    exit_option = 0.5
    utility_function = "normal"
    out_folder = "policy_comparison"
    num_repetitions = 1
    num_trials = 100
    context_dim = 5
    max_menu_size = 25
    previous_patients_per_provider = 10
    order="custom"
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--n_trials',          help='Number of trials ', type=int, default=2)
    parser.add_argument('--top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--true_top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--context_dim',          help='Context dim for patients and providers', type=int, default=5)
    parser.add_argument('--max_menu_size',          help='Context dim for patients and providers', type=int, default=50)
    parser.add_argument('--num_repetitions',          help='Context dim for patients and providers', type=int, default=1)
    parser.add_argument('--previous_patients_per_provider',          help='Context dim for patients and providers', type=int, default=10)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=5)
    parser.add_argument('--choice_model', help='Which choice model for patients', type=str, default='uniform_choice')
    parser.add_argument('--exit_option', help='What is the value of the exit option', type=float, default=0.5)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='random')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    provider_capacity = args.provider_capacity
    top_choice_prob = args.top_choice_prob
    choice_model = args.choice_model
    exit_option = args.exit_option
    out_folder = args.out_folder
    num_trials = args.n_trials 
    context_dim = args.context_dim 
    num_repetitions = args.num_repetitions
    true_top_choice_prob = args.true_top_choice_prob
    max_menu_size = args.max_menu_size
    utility_function = args.utility_function
    order = args.order
    previous_patients_per_provider = args.previous_patients_per_provider

save_name = secrets.token_hex(4)  

In [307]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'top_choice_prob': top_choice_prob, 
        'choice_model': choice_model,
        'exit_option': exit_option,
        'num_trials': num_trials,
        'context_dim': context_dim, 
        'true_top_choice_prob': true_top_choice_prob, 
        'num_repetitions': num_repetitions, 
        'max_menu_size': max_menu_size, 
        'utility_function': utility_function, 
        'order': order, 
        'previous_patients_per_provider': previous_patients_per_provider} 

## Baselines

In [300]:
seed_list = [seed]
restrict_resources()

In [301]:
policy = random_policy
name = "random"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_repetitions*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_repetitions*num_trials*len(seed_list))

random policy
Available providers [[93. 90. 86. 99. 89.]
 [95. 79. 83. 99. 91.]
 [94. 85. 78. 98. 91.]
 [94. 89. 85. 98. 93.]
 [94. 85. 78. 98. 93.]]
Selected [[ 1.  9. 13.  0.  3.]
 [ 3.  3. 11.  1.  3.]
 [ 3.  4.  1.  2.  4.]
 [ 5.  8.  9.  2.  2.]
 [ 4. 14.  4.  1.  9.]]
Prob [16. 38. 38.  6. 21.]
Took 0.01686239242553711 time


(0.238, 0.17849495358034292)

In [302]:
policy = all_ones_policy
name = "greedy_basic"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.mean(results['{}_minimums_all'.format(name)]),np.mean(results['{}_gaps_all'.format(name)]),np.mean(results['{}_variance_all'.format(name)])

greedy_basic policy
Available providers [[ 99.  78.  79. 100.  99.]
 [ 98.  77.  73. 100.  97.]
 [100.  81.  63. 100.  97.]
 [100.  80.  78. 100. 100.]
 [100.  82.  65. 100.  99.]]
Selected [[ 0.  4. 21.  0.  2.]
 [ 1.  3. 18.  0.  1.]
 [ 1. 12.  2.  0.  0.]
 [ 2.  6. 21.  0.  0.]
 [ 0. 28.  1.  0.  4.]]
Prob [ 4. 53. 63.  0.  7.]
Took 0.015822649002075195 time


(0.6374367699012243, 0.07273152217661365, 0.004956503958676469)

In [308]:
policy = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

greedy policy
Available providers [[ 84.  38.  38. 100.  89.]
 [ 86.  47.  37. 100.  84.]
 [ 93.  52.  31. 100.  82.]
 [ 95.  49.  43. 100.  85.]
 [ 76.  41.  24. 100.  84.]]
Selected [[ 5. 10. 32.  4. 30.]
 [10. 13. 28.  3. 21.]
 [30. 38.  1.  5.  1.]
 [27. 12. 35.  2.  2.]
 [ 4. 26.  3.  7. 27.]]
Prob [76. 99. 99. 21. 81.]
Took 0.024454832077026367 time


/usr0/home/naveenr/projects/patient_provider/patient/simulator.py:399: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  provider_workloads = [sum([len(j) for j in i])/len(i) for i in np.array(utilities_by_provider).T]
/usr0/home/naveenr/projects/patient_provider/patient/simulator.py:403: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  scores['matches'].append(num_matches)


(0.752, 0.5493837768444477)

In [20]:
if 2**(num_patients*num_providers)*2**(num_patients)*math.factorial(num_patients) < 100000:
    policy = one_shot_policy
    per_epoch_function = optimal_policy
    name = "optimal"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    results['{}_minimums'.format(name)] = rewards['provider_minimums']
    results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
    results['{}_gaps'.format(name)] = rewards['provider_gaps']
    results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
    results['{}_variance'.format(name)] = rewards['provider_variance']
    results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
    results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [237]:
policy = one_shot_policy
per_epoch_function = optimal_order_policy
name = "optimal_order"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

optimal_order policy
Took 0.017654895782470703 time
0.758 0.5405452364134852


## Offline

In [309]:
policy = one_shot_policy
per_epoch_function = lp_policy
name = "lp"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0))

lp policy
Available providers [[ 54.  62.  62.  71. 100.]
 [ 58.  64. 100.  72.  60.]
 [ 58. 100.  63.  73.  64.]
 [100.  67.  63.  77.  65.]
 [ 49.  56.  63. 100.  57.]]
Selected [[ 0.  0.  0.  0. 81.]
 [ 0.  0. 75.  0.  0.]
 [ 0. 75.  0.  0.  0.]
 [78.  0.  0.  0.  0.]
 [ 0.  0.  0. 67.  0.]]
Prob [78. 75. 75. 67. 81.]
Took 0.019945621490478516 time


(0.752, 0.5058918875744991, 6.507526357039188)

In [15]:
policy = one_shot_policy
per_epoch_function = lp_workload_policy
name = "lp_workload"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0))

lp_workload policy
Took 0.5431537628173828 time


(0.0, 0.0, 6.892960645691221, 6.892960645691221)

In [21]:
policy = one_shot_policy
per_epoch_function = lp_multiple_match_policy
name = "lp_multiple_match"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_multiple_match policy
Took 0.09562230110168457 time


(0.2536, 0.23916547111746667)

In [16]:
if choice_model == 'threshold':
    policy = one_shot_policy 
    per_epoch_function = lp_threshold
    name = "lp_threshold"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    results['{}_minimums'.format(name)] = rewards['provider_minimums']
    results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
    results['{}_gaps'.format(name)] = rewards['provider_gaps']
    results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
    results['{}_variance'.format(name)] = rewards['provider_variance']
    results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
    results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [17]:
policy = one_shot_policy 
per_epoch_function = lp_more_patients_policy
name = "lp_more_patients"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_more_patients policy
Took 0.6590728759765625 time


(0.7636, 0.5261014333470473)

In [18]:
policy = one_shot_policy
per_epoch_function = lp_fairness_policy
name = "lp_fairness"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_fairness policy
Took 0.5672309398651123 time


(0.7636, 0.4479570155876951)

In [19]:
policy = one_shot_policy
per_epoch_function = group_based_policy
name = "group_based"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

group_based policy
Took 0.715339183807373 time


(0.7636, 0.5261014333470473)

In [20]:
policy = one_shot_policy
per_epoch_function = group_based_unidirectional_policy
name = "group_based_unidirectional"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

group_based_unidirectional policy
Took 0.7381973266601562 time


(0.7504, 0.5187739882075151)

In [22]:
policy = one_shot_policy 
per_epoch_function = provider_focused_policy
name = "provider_focused"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

provider_focused policy
B 3, Per provider [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3.], Per Patient [1. 4. 1. 3. 4. 4. 3. 4. 2. 5. 1. 1. 2. 5. 2. 2. 3. 2. 4. 4. 5. 5. 1. 4.
 3.]
Took 0.1559138298034668 time


(0.6824, 0.511133744634547)

In [7]:
policy = one_shot_policy 
per_epoch_function = provider_focused_less_interference_policy
name = "provider_focused_less_interference"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

provider_focused_less_interference policy


NameError: name 'seed_list' is not defined

In [71]:
# policy = one_shot_policy 
# for lamb in [0.25,0.5,1,2,4]:
#     per_epoch_function = provider_focused_linear_regularization_policy(lamb)
#     name = "provider_focused_linear_regularization_{}".format(lamb)
#     print("{} policy".format(name))

#     rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

#     results['{}_matches'.format(name)] = rewards['matches']
#     results['{}_utilities'.format(name)] = rewards['patient_utilities']
#     results['{}_workloads'.format(name)] = rewards['provider_workloads']

#     results['{}_minimums'.format(name)] = rewards['provider_minimums']
#     results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
#     results['{}_gaps'.format(name)] = rewards['provider_gaps']
#     results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
#     results['{}_variance'.format(name)] = rewards['provider_variance']
#     results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
#     results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

#     print(lamb,np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

provider_focused_linear_regularization_0.25 policy
B 1, Per provider [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.], Per Patient [2. 2. 1. 1. 1. 0. 1. 0. 0. 1.]
Took 0.03953838348388672 time
0.25 0.7 0.49648776357158675
provider_focused_linear_regularization_0.5 policy
B 1, Per provider [1. 0. 1. 1. 1. 1. 1. 0. 1. 1.], Per Patient [1. 2. 1. 1. 1. 0. 1. 0. 0. 1.]


/usr0/home/naveenr/projects/patient_provider/patient/provider_policies.py:77: RuntimeWarning: invalid value encountered in double_scalars
  real_value += upper/lower*prod


Took 0.0397343635559082 time
0.5 0.7 0.49648776357158675
provider_focused_linear_regularization_1 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.031838178634643555 time
1 0.0 0.0
provider_focused_linear_regularization_2 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.03140616416931152 time
2 0.0 0.0
provider_focused_linear_regularization_4 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.03318023681640625 time
4 0.0 0.0


In [14]:
# policy = one_shot_policy 
# for lamb in [0,0.1,0.25,0.5]:#,1,2,4]:
#     per_epoch_function = provider_focused_log_regularization_policy(lamb)
#     name = "provider_focused_log_regularization_{}".format(lamb)
#     print("{} policy".format(name))

#     rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

#     results['{}_matches'.format(name)] = rewards['matches']
#     results['{}_utilities'.format(name)] = rewards['patient_utilities']
#     results['{}_workloads'.format(name)] = rewards['provider_workloads']

#     results['{}_minimums'.format(name)] = rewards['provider_minimums']
#     results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
#     results['{}_gaps'.format(name)] = rewards['provider_gaps']
#     results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
#     results['{}_variance'.format(name)] = rewards['provider_variance']
#     results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
#     results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

#     print(lamb,np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [13]:
# policy = one_shot_policy 
# per_epoch_function = gradient_descent_policy
# name = "gradient_descent"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# results['{}_matches'.format(name)] = rewards['matches']
# results['{}_utilities'.format(name)] = rewards['patient_utilities']
# results['{}_workloads'.format(name)] = rewards['provider_workloads']

# results['{}_minimums'.format(name)] = rewards['provider_minimums']
# results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
# results['{}_gaps'.format(name)] = rewards['provider_gaps']
# results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
# results['{}_variance'.format(name)] = rewards['provider_variance']
# results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
# results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

# print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [310]:
policy = one_shot_policy 
per_epoch_function = gradient_descent_policy_2
name = "gradient_descent_2"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

gradient_descent_2 policy
Available providers [[ 84.  38.  40. 100.  87.]
 [ 87.  47.  38. 100.  82.]
 [ 93.  52.  31. 100.  82.]
 [ 95.  49.  45. 100.  83.]
 [ 76.  41.  24. 100.  84.]]
Selected [[ 5. 10. 33.  4. 29.]
 [11. 13. 29.  3. 19.]
 [30. 38.  1.  5.  1.]
 [26. 12. 36.  2.  2.]
 [ 4. 26.  0.  7. 30.]]
Prob [76. 99. 99. 21. 81.]
Took 0.020916223526000977 time
0.752 0.5496695226211828


In [311]:
def objective2(z, theta, p, sorted_theta,lamb=1, smooth_reg='entropy', epsilon=1e-5):
    # Step 1: Compute sigmoid
    x = torch.sigmoid(z)

    # Step 2: Argsort `theta` and gather `x`
    argsorted = torch.argsort(theta, dim=1, descending=True)
    sorted_theta = theta.gather(1, argsorted)
    sorted_x = x.gather(1, argsorted)

    # Step 3: Compute rows_with_top_i
    rows_with_top_i = torch.cumsum((sorted_x > 0).float(), dim=0).transpose(0, 1)
    rows_with_top_i /= (theta.shape[0] - 1)

    # Step 4: Compute `is_top_k`
    is_top_k = torch.cumsum((sorted_x > 0).float() / (theta.shape[0] - 1), dim=1).transpose(0, 1)

    # Step 5: Normalize `x`
    S = theta.shape[0]
    scaled_x = torch.zeros_like(sorted_x)
    prod = torch.ones_like(sorted_x[:, :1])  # Start cumulative product with 1

    for top_num in range(S):
        delta = rows_with_top_i[:, top_num].unsqueeze(1) - is_top_k[:, :, top_num].transpose(0, 1)
        if top_num < S - 1:
            prod = prod * (1 - p * delta)
        scaled_x[:, top_num] = (1 / S) * prod.squeeze()

    # Scatter back scaled values to original order
    normalized_x = torch.zeros_like(x)
    normalized_x.scatter_(1, argsorted, scaled_x)

    # Step 6: Compute cumulative products for scaling
    one_minus_normalized_x = 1 - normalized_x
    cumprods = torch.cumprod(one_minus_normalized_x, dim=1)
    shifted_cumprods = torch.cat([torch.ones(cumprods.size(0), 1, device=cumprods.device), cumprods[:, :-1]], dim=1)
    scaled_normalized_x = normalized_x * shifted_cumprods

    # Final normalization
    normalized_x = scaled_normalized_x / torch.sum(scaled_normalized_x, dim=1, keepdim=True)

    # Step 7: Compute the loss
    term = p * torch.sum(normalized_x * theta, dim=0)
    term = torch.sum(term) / theta.shape[1]  # Normalize by the number of columns

    # Step 8: Add smooth regularization
    if smooth_reg == 'logit' and lamb > 0:
        reg_term = torch.sum(torch.logit(x, eps=epsilon) ** 2)  # Logit-based penalty
    elif smooth_reg == 'entropy' and lamb > 0:
        reg_term = -torch.sum(x * torch.log(x + epsilon) + (1 - x) * torch.log(1 - x + epsilon))  # Entropy-based penalty
    else:
        reg_term = 0

    # Final loss
    loss = term - lamb * reg_term

In [275]:
if is_jupyter:
    theta = [p.provider_rewards for p in simulator.patients]
    theta = torch.Tensor(theta)
    sorted_theta = torch.argsort(theta, dim=1,descending=True)  # Sorting indices of `theta` row-wise
    opt_tensor = torch.Tensor(lp_policy(simulator))
    ones_tensor = torch.Tensor(np.ones(opt_tensor.shape))
    x = torch.Tensor(gradient_descent_policy_2(simulator))
    p = true_top_choice_prob

In [312]:
if is_jupyter:
    print(objective2(ones_tensor*10000-10000/2,theta,p,sorted_theta,0))
    print(objective2(x*10000-10000/2,theta,p,sorted_theta,0))
    print(objective2(opt_tensor*10000-10000/2,theta,p,sorted_theta,0))

IndexError: too many indices for tensor of dimension 2

## Save Data

In [ ]:
save_path = get_save_path(out_folder,save_name)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)